In [ ]:
#run before bottom, only need to do once
!pip install selenium
!pip install beautifulsoup4
!pip install lxml

In [12]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as BS
import lxml
import csv

inputFile = "Classes.txt"
outputCSV = "Classes.csv"

#still some problems but works well enough

#getting to usc schedule page
#sign in manually
with webdriver.Chrome() as driver:
    driver.get("https://my.usc.edu")

    #was trying to sign in through selenium but realized it would be significantly easier with manual sign-in
    # if(driver.title.find("Single Sign-on")!=-1):
    #     print("Signing in")
    #     username = driver.find_element(By.NAME, "j_username")
    #     username.clear()
    #     username.send_keys("nqvo")
    #     password = driver.find_element(By.NAME, "j_password")
    #     password.clear()
    #     password.send_keys("Bubblesucks1234!")
    #     driver.find_element(By.NAME, "_eventId_proceed").click()

#waits until you're at the myUSC page
    while(driver.title.find("myUSC")==-1):
        True
    #goes to webreg
    driver.get("https://my.usc.edu/portal/oasis/webregbridge.php")
    #goes to the current semester
    driver.find_element(By.ID, "termLink1").click()
    assert "WebReg" in driver.title
    data = []
    try:
        #opens file with all the classes, codes should be like they are in the USC thing
        with open(inputFile, 'r') as file:
            content = file.read()
            #gets all the class codes
            codes = content.splitlines()
            for code in codes:
                # print(code)
                #splits the class code with the department code
                program = code[0:code.find("-")]
                #goes to the department site
                driver.get("https://webreg.usc.edu/Courses?Program="+program)
                #gets the html and soups it
                soup = BS(driver.page_source, 'lxml')
                #gets the current class on the site
                classe = soup.find("div", id="courseBin_"+code)
                if(classe):
                    #gets all the sections of the current class, formats them, gets their data, and puts them in a list of dictionary entries
                    classes = classe.find_all("div", class_="section")
                    for section in classes:
                        text = section.text
                        badInfo = text.splitlines()
                        goodInfo = []
                        for i in badInfo:
                            i = i.strip()
                            if i != "":
                                goodInfo.append(i)
                        print(goodInfo)
                        entry = {"Class:": code.strip()}
                        i = 1
                        while(i < len(goodInfo) - 1):
                            if(goodInfo[i].find(":") != len(goodInfo[i]) - 1):
                                entry[goodInfo[i-1]] = goodInfo[i]
                                i+=1
                            i+=1
                        print(entry)
                        data.append(entry)

                        # infoIndexes = {"section": 1, "type": 5, "time": 11, 13, 15, 17}
                        # if(goodInfo[infoPlaces] == "Instructor"):
                        # if(goodInfo[15] == "Location:"):
                        #     goodInfo[15] = ""
                        #     goodInfo[17] = goodInfo[16]
                        # if(goodInfo[17] == "Add to myCourseBin"):
                        #     goodInfo[17] = ""
                        # data.append({"class": code.strip(), "section": goodInfo[1], "type": goodInfo[5], "time": goodInfo[11], "days": goodInfo[13], "instructor": goodInfo[15], "location": goodInfo[17]})
    except FileNotFoundError:
        print("Error: The file 'your_file.txt' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

    fields = ["Class:", "Section:", "Session:", "Type:", "Units:", "Registered:", "Time:", "Days:", "Instructor:", "Location:"]

# writes the data into a csv
    with open(outputCSV, 'w') as csvfile:
        # creating a csv dict writer object
        writer = csv.DictWriter(csvfile, fieldnames=fields)

        # writing headers (field names)
        writer.writeheader()

        # writing data rows
        writer.writerows(data)

['Section:', '28700 D', 'Session:', '001', 'Type:', 'Lecture', 'Units:', '4.0', 'Registered:', '23 of 34', 'Time:', '08:00am-09:20am', 'Days:', 'TTh', 'Instructor:', 'Ronney, Paul', 'Location:', 'LVL16', 'Add to myCourseBin']
{'Class:': 'AME-101', 'Section:': '28700 D', 'Session:': '001', 'Type:': 'Lecture', 'Units:': '4.0', 'Registered:': '23 of 34', 'Time:': '08:00am-09:20am', 'Days:': 'TTh', 'Instructor:': 'Ronney, Paul', 'Location:': 'LVL16'}
['Section:', '28701 D', 'Session:', '001', 'Type:', 'Lecture', 'Units:', '4.0', 'Registered:', 'Closed', 'Time:', '09:30am-10:50am', 'Days:', 'TTh', 'Instructor:', 'Ronney, Paul', 'Location:', 'LVL16', 'Add to myCourseBin']
{'Class:': 'AME-101', 'Section:': '28701 D', 'Session:': '001', 'Type:': 'Lecture', 'Units:': '4.0', 'Registered:': 'Closed', 'Time:': '09:30am-10:50am', 'Days:': 'TTh', 'Instructor:': 'Ronney, Paul', 'Location:': 'LVL16'}
['Section:', '28702 R', 'Session:', '001', 'Type:', 'Lab', 'Units:', '0.0', 'Registered:', '26 of 33',